# --- Day 7: No Space Left On Device ---

## Part 1

For more detail, see https://adventofcode.com/2022/day/7

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

    $ system-update --please --pretty-please-with-sugar-on-top
    Error: No space left on device

Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:

To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. 



## My Approach

Examine the terminal output line by line.
Keep track of where you are (path), all the current directories including parents (indirs), and the accumulated size of all discovered directories (sizes).

If the line begins with \$, interpret as a command. There are two recognized commands, cd and ls.
* The cd command can navigate:
  * to the top
   * set the path to /
   * clear the list of parent directories
  * down one level
   * add the directory name to the path and the list of current directories
   * if this is a new directory, add it to the dictionary of discovered directories
  * up one level
   * remove the current path from the list of current directories 
   * remove the current directory from the end of the path
* The ls command is followed by multiple lines of data.
  * Examine the subsequent lines until you reach the end of the input or the next line begins with \$
  * If it's a directory, ignore
  * Otherwise assume it's a file and add the file size to all current directories, including parents. 
  
This builds a dictionary of all directories with their sizes. Cast it to a pandas dataframe to simplify selection and aggregation.


In [1]:
import re
import pandas as pd

sizes = {}   # accumulate sizes of known directories as you navigate the tree
indirs = []  # keep track of all the directories you are in, including ancestors
path = ''
    
with open('terminaloutput.txt', 'r') as f:
    term_out = f.read().split('\n')

while len(term_out) > 0:
    line = term_out.pop(0)

    if line.split()[0] == '$':
        if line.split()[1] == 'cd':
            if line.split()[2] == '..':
                indirs.remove(path)
                path = path[:[m.start() for m in re.finditer('/', path)].pop(-2)+1]
            else:
                if line.split()[2] =='/':
                    path = line.split()[2]
                    indirs = [path]
                else:
                    path += line.split()[2] + '/'
                    indirs.append(path)
                if not path in sizes:
                    sizes[path] = 0
                
        elif line.split()[1] == 'ls':
            while (len(term_out) > 0) and (not term_out[0].split()[0] == '$'):
                line = term_out.pop(0)
                if line.split()[0] != 'dir':
                    for d in indirs:
                        sizes[d] += int(line.split()[0])

df = pd.DataFrame.from_dict(sizes, orient='index', columns=['size'])

max100k = df[df['size'] <= 100000]

sum(max100k['size'])

1844187

## --- Part Two ---

Now, you're ready to choose a directory to delete.

The total disk space available to the filesystem is 70000000. To run the update, you need unused space of at least 30000000. You need to find a directory you can delete that will free up enough space to run the update.

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?


In [2]:
capacity = 70000000
update = 30000000

unused = capacity - sizes['/']

df.loc[df['size'] >= (update - unused), 'size'].min()

4978279